## XML Sitemap Generator

This notebook will generate all of the sitemaps that are stored within the Platform webapp ``/sitemap`` directory and linked to the [Platform index sitemap](https://www.targetvalidation.org/sitemaps/1804/index.xml). The index sitemap is the one that we submit to Google, Bing, Yahoo, Yandex, etc. to improve our SEO.

This notebook should be **run at least 3 times per year (every other release)** to ensure that search engines are correctly indexing the Platform. 

Links:

* [Current 
* [JS sitemap generator (stored in webapp repo)](https://github.com/opentargets/webapp/blob/master/sitemap-generator.js)
* [Creating XML sitemap from list](https://stackoverflow.com/questions/16681543/create-xml-file-with-python-by-iterating-over-lists)
* [ElementTree XML API documentation](https://docs.python.org/3.4/library/xml.etree.elementtree.html#building-xml-documents)
* [Sitemap XML format](https://www.sitemaps.org/protocol.html)